In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# read files for merge
df1 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 01.xlsx")
df2 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 02.xlsx")
df3 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 03.xlsx")
df4 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 04.xlsx")
df5 = pd.read_excel(r"C:\Users\karan\Downloads\Data - 05.xlsx")

In [ ]:
# merge files
df_merge = pd.concat([df1,df2,df3,df4,df5])
df_merge.head()

# EDA

In [ ]:
df_merge.shape

In [ ]:
df_merge.info()

In [ ]:
# check for missing values and remove
print(df_merge.isnull().sum())
df_merge.dropna(inplace=True)

In [ ]:
# check for duplicate values and remove
print(df_merge.duplicated().sum())
df_merge = df_merge.drop_duplicates(keep='first')

In [ ]:
df_merge.shape

# Preprocessing

In [ ]:
# replace unnecessary letters/words
replace_list = ["\\n","\\t","\\r","â","€","™","ð","Ÿ","ðŸ‘","$","â€™ll","ƒ","¢","â€ƒ","â€¢","Â§","§","Â","Ã¼","Ã","¼","º","œ","˜","£","â€“","â€œ","&lt;#&gt;","â€Œ","ðŸŽ","ð","Ÿ","Ž","Í","â€Œ ï»¿ Í","ï»¿","ðŸ”¨","ðŸ¤©","©","¤","±","ðŸ˜±","ðŸ˜±"," Í â€Œ ï»¿","ÿ" ]
for letter in replace_list:
    df_merge['Text'] = df_merge['Text'].str.replace(letter, '')
print(df_merge.head())

In [ ]:
# function for preprocessing
def preprocessing(text):
    if isinstance(text, str):
        text = text
        text = nltk.word_tokenize(text)

        filtered_words = []
        for word in text:
            if word.isalnum():
                filtered_words.append(word)

        text = filtered_words[:]
        filtered_words.clear()

        for word in text:
            if word not in stopwords.words('english') and word not in string.punctuation:
                filtered_words.append(word)

        text = filtered_words[:]
        filtered_words.clear()

        for word in text:
            filtered_words.append(ps.stem(word))

        return " ".join(filtered_words)
    else:
        # convert to string
        return str(text)

In [ ]:
df_merge.head()